# Data Processing

In [1]:
from pathlib import Path
import pandas as pd
import re
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import json

from processing_utils import xyz_to_df, pairwise_distance, average_abs_dist, xyz_to_np

HARTREE_TO_KCAL_MOL = 627.509

In [2]:
experiments_path = Path("experiments")
processed_results_path = Path("processed_results")
molecules_path = Path("uniques_100_molecules_42_seed")

## Parse Results

### Parse Experiment Results from xtb Output Files

In [3]:
results_df = pd.DataFrame(
    columns=[
        "molecule",
        "parameter",
        "factor",
        "parameter value",
        "atomisation energy (Hartrees)",
        "atomisation energy (kcal/mol)",
        "total energy (Hartrees)",
        "total energy (kcal/mol)",
        "pairwise distance",
        "atoms",
    ]
)

for parameter_path in tqdm(
    experiments_path.glob("*"), total=len(list(experiments_path.glob("*")))
):
    for run_path in parameter_path.glob("*"):
        parameter_factor, parameter_value = run_path.name.split("_")
        num_skipped = 0
        for molecule in run_path.glob("*"):
            if not molecule.is_dir():
                continue

            # Atomisation energy
            with open(molecule / "stdout.txt") as f:
                output_file = f.readlines()
            total_energy, atomisation_energy = None, None
            for line in output_file:
                if total_energy is not None and atomisation_energy is not None:
                    break
                elif "total energy" in line:
                    total_energy = float(re.sub(r"[^\d.]", "", line))
                elif "atomisation energy" in line:
                    atomisation_energy = float(re.sub(r"[^\d.]", "", line))

            # Geometry
            pairwise_distances_arr = None
            if (molecule / "xtbopt.xyz").exists():
                geometry_arr, atoms = xyz_to_np(molecule / "xtbopt.xyz")
                pairwise_distances_arr = pairwise_distance(geometry_arr)

            if (
                atomisation_energy is None
                or total_energy is None
                or pairwise_distances_arr is None
            ):
                num_skipped += 1
                continue

            results_df.loc[len(results_df)] = [
                molecule.name,
                parameter_path.name,
                parameter_factor,
                parameter_value,
                atomisation_energy,
                atomisation_energy * HARTREE_TO_KCAL_MOL,
                total_energy,
                total_energy * HARTREE_TO_KCAL_MOL,
                json.dumps(pairwise_distances_arr.tolist()),
                atoms,
            ]
        print(
            f"Skipped {num_skipped} / {len(list(run_path.glob('*')))} molecules in {parameter_path.name} with parameter {parameter_factor} and value {parameter_value}."
        )

  0%|          | 0/24 [00:00<?, ?it/s]

Skipped 9 / 99 molecules in kexplight with parameter 1.2 and value 1.2.
Skipped 11 / 99 molecules in kexplight with parameter 1.1 and value 1.1.
Skipped 6 / 99 molecules in kexplight with parameter 1.0 and value 1.0.
Skipped 6 / 99 molecules in kexplight with parameter 0.9 and value 0.9.


  4%|▍         | 1/24 [00:02<00:46,  2.03s/it]

Skipped 8 / 99 molecules in kexplight with parameter 0.8 and value 0.8.
Skipped 9 / 99 molecules in gam3p with parameter 0.8 and value 0.4.
Skipped 10 / 99 molecules in gam3p with parameter 1.1 and value 0.55.
Skipped 13 / 99 molecules in gam3p with parameter 1.2 and value 0.6.
Skipped 11 / 99 molecules in gam3p with parameter 0.9 and value 0.45.


  8%|▊         | 2/24 [00:03<00:43,  1.96s/it]

...

Skipped 12 / 99 molecules in ipeashift with parameter 0.9 and value 1.602621.
Skipped 15 / 99 molecules in aesdmp5 with parameter 0.8 and value 3.2.
Skipped 12 / 99 molecules in aesdmp5 with parameter 1.2 and value 4.8.
Skipped 11 / 99 molecules in aesdmp5 with parameter 1.0 and value 4.0.
Skipped 14 / 99 molecules in aesdmp5 with parameter 0.9 and value 3.6.


 96%|█████████▌| 23/24 [00:49<00:02,  2.37s/it]

Skipped 13 / 99 molecules in aesdmp5 with parameter 1.1 and value 4.4.
Skipped 13 / 99 molecules in aesexp with parameter 0.8 and value 3.2.
Skipped 12 / 99 molecules in aesexp with parameter 1.2 and value 4.8.
Skipped 11 / 99 molecules in aesexp with parameter 1.0 and value 4.0.
Skipped 12 / 99 molecules in aesexp with parameter 0.9 and value 3.6.


100%|██████████| 24/24 [00:51<00:00,  2.14s/it]

Skipped 12 / 99 molecules in aesexp with parameter 1.1 and value 4.4.


In [4]:
# to pickle
results_df.to_csv(processed_results_path / "parsed_results.csv", index=False)

### Add VQM24 Reference

In [5]:
results_df = pd.read_csv(processed_results_path / "parsed_results.csv")

results_df["pairwise distance"] = results_df["pairwise distance"].apply(lambda x: np.array(json.loads(x)))

reference_data = np.load("reference_data/DFT_uniques.npz", allow_pickle=True)

In [6]:
results_df["VQM24 atomisation energy (Hartrees)"] = None
results_df["VQM24 atomisation energy (kcal/mol)"] = None
results_df["VQM24 total energy (Hartrees)"] = None
results_df["VQM24 total energy (kcal/mol)"] = None

results_df["VQM24 pairwise distance"] = None

for molecule in tqdm(
    results_df["molecule"].unique(),
    total=len(results_df["molecule"].unique()),
    desc="Adding VQM24 data",
):
    # reference compounds use SMIELS_num/conformer_num
    # results compounds use SMIELS_num_conformer_num
    vqm24_molecule = molecule.replace("_", "/", 2)
    vqm24_molecule = vqm24_molecule.replace("/", "_", 1)

    reference_index = np.where(reference_data["compounds"] == vqm24_molecule)

    # VQM24 energies are flipped
    reference_atomization_energy = -reference_data["Eatomization"][
        reference_index
    ].item()
    reference_total_energy = -reference_data["Etot"][reference_index].item()

    results_df.loc[
        results_df["molecule"] == molecule, "VQM24 atomisation energy (Hartrees)"
    ] = reference_atomization_energy
    results_df.loc[
        results_df["molecule"] == molecule, "VQM24 atomisation energy (kcal/mol)"
    ] = (reference_atomization_energy * HARTREE_TO_KCAL_MOL)

    results_df.loc[
        results_df["molecule"] == molecule, "VQM24 total energy (Hartrees)"
    ] = (reference_total_energy / HARTREE_TO_KCAL_MOL)
    results_df.loc[
        results_df["molecule"] == molecule, "VQM24 total energy (kcal/mol)"
    ] = reference_total_energy

    # Geometries
    xyz_df = molecules_path / f"{molecule}.xyz"
    pairwise_distances_arr = pairwise_distance(xyz_to_np(xyz_df)[0])

    mask = results_df["molecule"] == molecule

    results_df.loc[mask, "VQM24 pairwise distance"] = [
        json.dumps(pairwise_distances_arr.tolist())
    ] * mask.sum()

results_df['VQM24 pairwise distance'] = results_df['VQM24 pairwise distance'].apply(lambda x: np.array(json.loads(x)))

results_df["Atomization Energy MAE (Hartrees)"] = abs(
    results_df["atomisation energy (Hartrees)"]
    - results_df["VQM24 atomisation energy (Hartrees)"]
)
results_df["Atomization Energy MAE (kcal/mol)"] = abs(
    results_df["atomisation energy (kcal/mol)"]
    - results_df["VQM24 atomisation energy (kcal/mol)"]
)

results_df["Total Energy MAE (Hartrees)"] = abs(
    results_df["total energy (Hartrees)"] - results_df["VQM24 total energy (Hartrees)"]
)
results_df["Total Energy MAE (kcal/mol)"] = abs(
    results_df["total energy (kcal/mol)"] - results_df["VQM24 total energy (kcal/mol)"]
)

results_df["Pairwise Distance MAE"] = results_df.apply(
    lambda row: average_abs_dist(
        row["pairwise distance"], row["VQM24 pairwise distance"]
    ),
    axis=1,
)

results_df["Pairwise Distance Frobenius Norm"] = results_df.apply(
    lambda row: np.linalg.norm(
        row["pairwise distance"] - row["VQM24 pairwise distance"]
    ),
    axis=1,
)

Adding VQM24 data: 100%|██████████| 96/96 [00:18<00:00,  5.10it/s]


In [7]:
results_df.head()

,molecule,parameter,factor,parameter value,atomisation energy (Hartrees),atomisation energy (kcal/mol),total energy (Hartrees),total energy (kcal/mol),pairwise distance,atoms,...,VQM24 atomisation energy (kcal/mol),VQM24 total energy (Hartrees),VQM24 total energy (kcal/mol),VQM24 pairwise distance,Atomization Energy MAE (Hartrees),Atomization Energy MAE (kcal/mol),Total Energy MAE (Hartrees),Total Energy MAE (kcal/mol),Pairwise Distance MAE,Pairwise Distance Frobenius Norm
0,BrSSiSxH3_2,kexplight,1.2,1.2,0.707659,444.062183,13.650865,8566.040903,"[[0.0, 2.447864695485579, 3.8423510933777245, ...","['S', 'Si', 'Br', 'S', 'H', 'H', 'H']",...,446.295146,5.835438,3661.789619,"[[0.0, 2.6040135127791433, 3.523605859355621, ...",0.003558,2.232964,7.815428,4904.251284,0.697765,6.242887
1,C2ClFSxH4_22,kexplight,1.2,1.2,1.613221,1012.310525,18.675790,11719.226458,"[[0.0, 1.3390988712738194, 2.708443513153894, ...","['F', 'C', 'S', 'Cl', 'C', 'H', 'H', 'H', 'H']",...,693.249515,1.652139,1036.731911,"[[0.0, 1.333680567403643, 2.7466757603578444, ...",0.508456,319.06101,17.023652,10682.494547,0.058493,0.801000
2,BrO2P2H3_8,kexplight,1.2,1.2,1.548242,971.535771,18.770090,11778.400337,"[[0.0, 1.6480754808629658, 2.5886844666025537,...","['P', 'O', 'O', 'P', 'Br', 'H', 'H', 'H']",...,488.052273,5.432418,3408.891139,"[[0.0, 1.6994959608214022, 2.596585491161673, ...",0.770481,483.483497,13.337672,8369.509198,0.304710,3.678716
3,ClOP3H0_1,kexplight,1.2,1.2,1.216244,763.204064,16.157681,10139.090245,"[[0.0, 1.457927102332756, 2.977110745100965, 4...","['O', 'P', 'P', 'P', 'Cl']",...,312.381174,2.485124,1559.437989,"[[0.0, 1.5071483694377603, 3.110195158460032, ...",0.718433,450.82289,13.672556,8579.652256,0.293298,2.366234
4,CN2PSH1_54,kexplight,1.2,1.2,1.767130,1108.890292,14.221133,8923.888752,"[[0.0, 1.6659726863761082, 2.466966445470851, ...","['N', 'P', 'N', 'S', 'C', 'H']",...,531.570446,1.414629,887.692235,"[[0.0, 1.702211789424578, 2.5644599381846733, ...",0.920018,577.319846,12.806504,8036.196517,0.032146,0.230476


In [8]:
results_df.to_csv(processed_results_path / "with_reference_results.csv", index=False)

## Aggregate Results

In [9]:
results_df = pd.read_csv(processed_results_path / "with_reference_results.csv")

In [10]:
aggregated_results = (
    results_df.groupby(["parameter", "factor", "parameter value"])[[
        "Atomization Energy MAE (kcal/mol)",
        "Total Energy MAE (kcal/mol)",
        "Pairwise Distance MAE",
        "Pairwise Distance Frobenius Norm",
    ]]
    .mean()
    .reset_index()
)
aggregated_results.head()

,parameter,factor,parameter value,Atomization Energy MAE (kcal/mol),Total Energy MAE (kcal/mol),Pairwise Distance MAE,Pairwise Distance Frobenius Norm
0,a1,0.8,0.416,447.168610,8436.629131,0.402464,5.341198
1,a1,0.9,0.468,465.945616,8384.836193,0.368220,4.822834
2,a1,1.0,0.520,450.541897,8402.476356,0.383720,5.053570
3,a1,1.1,0.572,467.470238,8385.269437,0.370357,4.830669
4,a1,1.2,0.624,456.347838,8383.214394,0.381624,5.071695


In [11]:
aggregated_results.to_csv(
    processed_results_path / "aggregated_results.csv", index=False
)